# Quantum Annealing Method for Dynamic Virtual Machine and Task Allocation in Cloud Infrastructures from Sustainability Perspective

Valter Uotila, Jiaheng Lu

Unified Database Management Systems research group

Department of Computer Science

University of Helsinki

Finland

The initial version of this work won the Most Creative Open Hackathon Experiment on Braket in Amazon Braket Challenge and Third place in IBM Qiskit Challenge at the QHack quantum computing hackathon organized by Xanadu in the spring of 2022.

In [1]:
import dimod
from dimod.generators.constraints import combinations
from dwave.system import LeapHybridSampler
from hybrid.reference import KerberosSampler

import itertools
import math
import json

from flask import Flask
from flask import request

## Tasks and task dependencies

We develop the model so that we are able to optimize over multiple objectives. Anyway, one of the most comprehensive cloud simulators, CloudSim, does not easily allow control over all of these objectives. For example, in PlanetLab workload the system creates as many virtual machines as there are workloads. The allocation of tasks to virtual machines is pratically random which is not optimal. On the other hand, the system does not allow changes to this allocation during the simulation.

In pratice we can optimize the initial task to virtual machine allocation and also virtual machine to physical machine (host) allocation. This is the first optimization round. We model the system as a single datacenter. Including multiple datacenters would not make the approach different except that bandwith between the datacenters must be more limited than bandwith in a single datacenter. These single datacenter simulations also seem to be the standard in the field.

After the initial optimization round, we do not change task-virtual machine allocation because it is not possible in the simulator without extensive modifications to the source code. Instead, we start optimizing virtual machine - physcial machine allocation with respect to sustainability, budget and time. The optimization is performed in intervals. The length of an interval is defined by the user.

In [2]:
def construct_variables(datacenter_state, previous_state = []):
    variables = set()
    for task in datacenter_state["cloudlets"]:
        for vm in datacenter_state["vms"]:
            if previous_state != []:
                for var in previous_state:
                    if int(task) == int(var[0]) and int(vm) == int(var[1]):
                        for pm in datacenter_state["hosts"]:
                            variables.add((int(task), int(vm), int(pm)))
            else:
                for pm in datacenter_state["hosts"]:
                    variables.add((int(task), int(vm), int(pm)))
    return list(variables)

In [3]:
def group_by(variables, index):
    groups = {}
    for var in variables:
        if var[index] in groups:
            groups[var[index]].append(var)
        else:
            groups[var[index]] = [var]
    return groups

In [4]:
def construct_previous_state(datacenter_state, first_run):
    vm_pm_state = []
    state = []
    if not first_run:
        for vm in datacenter_state["vms"]:
            vm_pm_state.append((vm, datacenter_state["vms"][vm]["hostid"]))
        for cloudlet in datacenter_state["cloudlets"]:
            cloudlet_id = datacenter_state["cloudlets"][cloudlet]["cloudletId"]
            cloudlet_vm = datacenter_state["cloudlets"][cloudlet]["vmId"]
            for vm_pm in vm_pm_state:
                if int(vm_pm[0]) == int(cloudlet_vm):
                    state.append((cloudlet_id, vm_pm[0], vm_pm[1]))
    return state

## Metrics

These metrics can be modified based on the cloud infrastructure.

In [5]:
def price(t, v, datacenter_state):
    return 0

In [6]:
def get_task_mips(task_id, datacenter_state):
    return datacenter_state["cloudlets"][str(task_id)]["totalLength"]

def get_vm_mips(vm_id, datacenter_state):
    return datacenter_state["vms"][str(vm_id)]["mips"]

def get_pm_mips(pm_id, datacenter_state):
    return datacenter_state["hosts"][str(pm_id)]["mips"]

def get_pm_power(pm_id, datacenter_state):
    return datacenter_state["hosts"][str(pm_id)]["power"]

def get_vm_bw(vm_id, datacenter_state):
    return datacenter_state["vms"][str(vm_id)]["bw"]

def get_pm_bw(pm_id, datacenter_state):
    return datacenter_state["hosts"][str(pm_id)]["availableBw"]

def get_sustainability_scaling_value(pm_id):
    return pm_id**2

def get_sustainability_offset(pm_id):
    return pm_id**3

Running virtual machine on a physical machine requires a certain amount of resources. The power consumption is modelled depending on the ratio (required resouces / total resources). Furthermore, the sustainability is modelled as a function where part of the carbon footprint is related to the source of the power production and an additional constant offset which encodes the carbon footprint related to the water consumption (part of cooling) and other construction related consumptions.

In [7]:
def sustainability(t, v, m, datacenter_state):
    # (VM mips / PM MIPS)*max power for PM*sustainability scaling value for power production + carbon footprint offset
    power =  get_pm_power(m, datacenter_state)
    v_mips = get_vm_mips(v, datacenter_state)
    m_mips = get_pm_mips(m, datacenter_state)
    
    scaling_value = get_sustainability_scaling_value(m)
    offset = get_sustainability_offset(m)
    
    carbon_footprint = (v_mips / m_mips)*power*scaling_value + offset
    
    return carbon_footprint

In [8]:
def migration_cost(to_var, from_var, datacenter_state):
    # Determining migration cost is a bit open question in the implementation
    # Energy and carbon footprint
    # Time and resources
    # VM size / available bandwidth
    
    cost = 0
    vm_bw = get_vm_bw(from_var[1], datacenter_state)
    pm_bw = get_pm_bw(to_var[2], datacenter_state)
    
    if vm_bw > pm_bw:
        cost = 1000000
    else:
        cost = pm_bw - vm_bw
    
    return cost

## Constraints

### Hard constraints

Violation of these constraints is not technically possible and these constraints should have high priority in order that the solution is even valid. These constraints are forced to be true by adjust the `strength` constant.

#### Constraint: every task executed on a single virtual machine

In [9]:
def every_task_executed_on_single_vm(variables, bqm):
    strength = 10000000
    groups = group_by(variables, 0)
    for group in groups:
        combs = combinations(groups[group], 1, strength)
        bqm.update(combs)
    return bqm

#### Constraint: every virtual machine runs on single physical machine

In [10]:
def every_vm_runs_on_single_pm(variables, bqm):
    strength = 10000000
    groups = group_by(variables, 1)
    for group in groups:
        combs = combinations(groups[group], 1, strength)
        bqm.update(combs)
    return bqm

### Soft constraints

Violation of these constraints is possible but not desired. The mimization of these constraints makes the solution more optimal.

#### Constraint: The available physical machines should be utilized efficiently

This step aims to optimize the virtual machine allocation by allocating virtual machines to physical machines so that the execution time is minimized. The execution time is estimated by considering task sizes on virtual machines in MIPS and considering virtual machine sizes on physical machines in MIPS. When the physical resources are utilized efficiently, the running time is minimized.

Since cloudlet MIPS > VM MIPS, we aim to position cloudlets to virtual machines so that cloudlets' total running time is minimized.

In [11]:
def maximize_vm_utilization(variables, bqm, datacenter_state):
    vms = group_by(variables, 1)
    linear = dict()
    quadratic = dict()
    offset = 0.0
    
    for vm in vms:
        total_for_vm = get_vm_mips(vm, datacenter_state)
        for var in vms[vm]:
            task = var[0]
            total_for_task = get_task_mips(task, datacenter_state)
            task_running_time_on_vm = total_for_task / total_for_vm
            linear[var] = task_running_time_on_vm
            
    bqm_migration = dimod.BinaryQuadraticModel(linear, quadratic, offset, dimod.BINARY)
    bqm.update(bqm_migration)
    return bqm

#### Constraint: The available virtual machines should be utilized efficiently 

It is important that VM MIPS < PM MIPS. If the condition is not satisfied, no VM is allocated to a PM.

In [12]:
def maximize_pm_utilization(variables, bqm, datacenter_state):
    pms = group_by(variables, 2)
    linear = dict()
    quadratic = dict()
    offset = 0.0
    
    for p in pms:
        quadratic_terms = itertools.combinations(pms[p], 2)
        total_for_this_pm = get_pm_mips(p, datacenter_state)
        offset = total_for_this_pm**2

        # Quadratic terms
        for t in quadratic_terms:
            vm1, vm2 = t[0][1], t[1][1]
            quadratic[t] = 2*get_vm_mips(vm1, datacenter_state)*get_vm_mips(vm2, datacenter_state)

        # Linear terms
        for var in pms[p]:
            vm = var[1]
            linear[var] = get_vm_mips(vm, datacenter_state)**2 - 2*total_for_this_pm
    
    bqm_migration = dimod.BinaryQuadraticModel(linear, quadratic, offset, dimod.BINARY)
    bqm.update(bqm_migration)
    return bqm

#### Constraint: migration cost between machines should be minimized

Virtual machine allocations after the initial step can be migrated between the physical machines in the cloud infrastructure.

In [13]:
def minimize_migration_cost(variables, previous_state, bqm, datacenter_state):
    linear = dict()
    quadratic = dict()
    to_vms = group_by(variables, 1)
    from_vms = previous_state

    for from_vm in from_vms:
        for to_vm in to_vms:
            for to_vm_var in to_vms[to_vm]:
                linear[to_vm_var] = migration_cost(to_vm_var, from_vm, datacenter_state)
    
    bqm_migration = dimod.BinaryQuadraticModel(linear, quadratic, 0.0, dimod.BINARY)
    bqm.update(bqm_migration)
    return bqm

#### Constraint: sustainability should be maximized

In [14]:
def maximize_sustainability(variables, bqm, datacenter_state):
    linear = dict()
    for var in variables:
        linear[var] = sustainability(var[0], var[1], var[2], datacenter_state)
        
    bqm_migration = dimod.BinaryQuadraticModel(linear, dict(), 0.0, dimod.BINARY)
    bqm.update(bqm_migration)
    return bqm

## Optimization on quantum annealer

In [15]:
def initialize():
    vartype = dimod.BINARY
    linear = dict()
    quadratic = dict()
    offset = 0.0
    bqm = dimod.BinaryQuadraticModel(linear, quadratic, offset, vartype)
    return bqm

In [16]:
def solve(bqm, device = "Simulated"):
    sample = None
    bqm.normalize()
    if device == "Simulated":
        sampler = dimod.SimulatedAnnealingSampler()
        sampleset = sampler.sample(bqm, num_reads=1000)
        sample = sampleset.first.sample
    elif device == "Kerberos":
        kerberos_sampler = KerberosSampler().sample(bqm, max_iter=10, convergence=3, qpu_params={'label': 'Task and VM allocation'})
        sample = kerberos_sampler.first.sample
    elif device == "LeapHybrid":
        sampler = LeapHybridSampler()
        sampleset = sampler.sample(bqm)
        sample = sampleset.first.sample
    return sample

In [17]:
def print_solution(sample):
    for varname, value in sample.items():
        if value == 1:
            print(varname, value)

In [18]:
def construct_migration_map(solution):
    migration_map = {}
    for varname, value in solution.items():
        if value == 1:
            migration_map[str(varname[1])] = str(varname[2]) 
    return migration_map

In [19]:
def construct_task_allocation_map(solution):
    task_allocation_map = {}
    for varname, value in solution.items():
        if value == 1:
            task_allocation_map[str(varname[0])] = str(varname[1]) 
    return task_allocation_map

In [20]:
app = Flask(__name__)

@app.route("/", methods=['POST'])
def optimize_allocation():
    datacenter_state = request.get_json()
    time = datacenter_state["time"]
    interval = datacenter_state["interval"]
    first_run = time == 0.0
    
    previous_state = construct_previous_state(datacenter_state, first_run)
    
    variables = construct_variables(datacenter_state, previous_state)
    
    bqm = initialize()
    
    print("The problem contains", len(variables), "variables.")
    
    print("Constructing qubo...")
    bqm = every_task_executed_on_single_vm(variables, bqm)
    bqm = every_vm_runs_on_single_pm(variables, bqm)
    bqm = maximize_pm_utilization(variables, bqm, datacenter_state)
    bqm = maximize_sustainability(variables, bqm, datacenter_state)
    
    # We optimize the task allocation on virtual machines on the first optimization round
    # After that we minimize migration cost of virtual machines
    if first_run:
        bqm = maximize_vm_utilization(variables, bqm, datacenter_state)
    else:
        bqm = minimize_migration_cost(variables, previous_state, bqm, datacenter_state)
    
    print("Number of quadratic terms", len(bqm.quadratic))
    
    device = "LeapHybrid"
    print("Solving QUBO on", device)
    #solution = solve(bqm, device)
    #print_solution(solution)
    
    #response = construct_migration_map(solution)
    response = {}
    for v in datacenter_state["vms"]:
        response[v] = v
    return response

In [ ]:
app.run(port=1234)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:1234/ (Press CTRL+C to quit)


[]
[(10, 5, 11), (2, 8, 3), (2, 11, 8), (14, 14, 1), (19, 15, 19), (11, 18, 11), (1, 7, 13), (5, 16, 3), (18, 0, 7), (13, 5, 8), (2, 4, 5), (19, 8, 16), (11, 14, 13), (6, 7, 8), (5, 9, 0), (18, 12, 17), (5, 8, 4), (5, 11, 9), (2, 15, 19), (1, 18, 0), (6, 19, 18), (11, 6, 14), (5, 1, 1), (17, 9, 4), (8, 8, 1), (11, 9, 19), (5, 4, 6), (2, 8, 16), (17, 18, 9), (14, 14, 14), (17, 2, 1), (8, 4, 3), (1, 6, 3), (17, 13, 15), (5, 8, 17), (0, 2, 1), (1, 18, 13), (17, 6, 12), (14, 2, 17), (1, 17, 17), (5, 1, 14), (9, 12, 9), (17, 9, 17), (5, 4, 19), (13, 15, 12), (12, 17, 4), (1, 10, 14), (13, 18, 17), (1, 13, 19), (17, 1, 18), (9, 4, 10), (13, 8, 9), (9, 7, 15), (13, 11, 14), (0, 6, 12), (1, 6, 16), (12, 12, 10), (4, 15, 2), (8, 19, 1), (9, 0, 12), (13, 4, 11), (15, 17, 5), (0, 2, 14), (12, 5, 7), (8, 14, 7), (7, 15, 3), (12, 0, 13), (4, 3, 5), (8, 7, 4), (16, 9, 3), (15, 5, 8), (7, 8, 0), (17, 17, 12), (7, 11, 5), (4, 15, 15), (16, 18, 8), (16, 2, 0), (3, 17, 0), (10, 16, 0), (8, 2, 10), (0, 5

127.0.0.1 - - [04/Jan/2023 13:54:42] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:42] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:43] "POST / HTTP/1.1" 200 -


Number of quadratic terms 4560000
Solving QUBO on LeapHybrid
[(0, '0', 1), (1, '1', 1), (2, '2', 3), (3, '3', 3), (4, '4', 5), (5, '5', 5), (6, '6', 7), (7, '7', 7), (8, '8', 9), (9, '9', 9), (11, '11', 0), (12, '12', 2), (13, '13', 2), (14, '14', 4), (15, '15', 0), (16, '16', 2), (17, '17', 4), (18, '18', 4), (19, '19', 4)]
[(13, 13, 4), (8, 8, 6), (13, 13, 13), (14, 14, 1), (8, 8, 15), (14, 14, 10), (14, 14, 19), (5, 5, 3), (5, 5, 12), (19, 19, 7), (19, 19, 16), (9, 9, 4), (17, 17, 9), (12, 12, 11), (4, 4, 3), (17, 17, 18), (4, 4, 12), (3, 3, 4), (3, 3, 13), (8, 8, 1), (13, 13, 8), (8, 8, 10), (14, 14, 5), (14, 14, 14), (5, 5, 7), (19, 19, 2), (19, 19, 11), (7, 7, 18), (17, 17, 4), (12, 12, 6), (17, 17, 13), (12, 12, 15), (4, 4, 7), (4, 4, 16), (3, 3, 8), (18, 18, 19), (3, 3, 17), (8, 8, 5), (14, 14, 0), (14, 14, 9), (14, 14, 18), (6, 6, 6), (6, 6, 15), (7, 7, 13), (12, 12, 1), (17, 17, 8), (12, 12, 10), (4, 4, 2), (17, 17, 17), (18, 18, 5), (12, 12, 19), (4, 4, 11), (3, 3, 3), (18, 

...

[(0, '0', 0), (1, '1', 1), (2, '2', 2), (3, '3', 3), (19, '19', 19), (5, '5', 5), (8, '8', 8), (11, '11', 11), (13, '13', 13), (15, '15', 15)]
[(19, 19, 14), (1, 1, 16), (13, 13, 4), (8, 8, 6), (13, 13, 13), (2, 2, 5), (8, 8, 15), (2, 2, 14), (11, 11, 5), (5, 5, 3), (3, 3, 2), (5, 5, 12), (15, 15, 7), (3, 3, 11), (19, 19, 7), (0, 0, 7), (1, 1, 0), (19, 19, 16), (15, 15, 16), (0, 0, 16), (1, 1, 9), (13, 13, 6), (1, 1, 18), (8, 8, 8), (13, 13, 15), (2, 2, 7), (8, 8, 17), (2, 2, 16), (5, 5, 5), (3, 3, 4), (19, 19, 0), (0, 0, 0), (3, 3, 13), (19, 19, 9), (15, 15, 9), (0, 0, 9), (1, 1, 2), (19, 19, 18), (15, 15, 18), (0, 0, 18), (1, 1, 11), (8, 8, 1), (13, 13, 8), (2, 2, 0), (8, 8, 10), (2, 2, 9), (0, 0, 14), (5, 5, 7), (15, 15, 2), (3, 3, 6), (19, 19, 2), (0, 0, 2), (3, 3, 15), (19, 19, 11), (15, 15, 11), (1, 1, 4), (0, 0, 11), (13, 13, 1), (8, 8, 3), (2, 2, 2), (8, 8, 12), (2, 2, 11), (5, 5, 0), (15, 15, 4), (3, 3, 8), (19, 19, 4), (11, 11, 14), (0, 0, 4), (3, 3, 17), (15, 15, 13), (0, 0,

127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -



[(19, 19, 14), (1, 1, 16), (13, 13, 4), (8, 8, 6), (13, 13, 13), (2, 2, 5), (8, 8, 15), (2, 2, 14), (11, 11, 5), (5, 5, 3), (3, 3, 2), (5, 5, 12), (15, 15, 7), (3, 3, 11), (19, 19, 7), (0, 0, 7), (1, 1, 0), (19, 19, 16), (15, 15, 16), (0, 0, 16), (1, 1, 9), (13, 13, 6), (1, 1, 18), (8, 8, 8), (13, 13, 15), (2, 2, 7), (8, 8, 17), (2, 2, 16), (5, 5, 5), (3, 3, 4), (19, 19, 0), (0, 0, 0), (3, 3, 13), (19, 19, 9), (15, 15, 9), (0, 0, 9), (1, 1, 2), (19, 19, 18), (15, 15, 18), (0, 0, 18), (1, 1, 11), (8, 8, 1), (13, 13, 8), (2, 2, 0), (8, 8, 10), (2, 2, 9), (0, 0, 14), (5, 5, 7), (15, 15, 2), (3, 3, 6), (19, 19, 2), (0, 0, 2), (3, 3, 15), (19, 19, 11), (15, 15, 11), (1, 1, 4), (0, 0, 11), (13, 13, 1), (8, 8, 3), (2, 2, 2), (8, 8, 12), (2, 2, 11), (5, 5, 0), (15, 15, 4), (3, 3, 8), (19, 19, 4), (11, 11, 14), (0, 0, 4), (3, 3, 17), (15, 15, 13), (0, 0, 13), (8, 8, 5), (2, 2, 4), (3, 3, 1), (11, 11, 7), (15, 15, 6), (3, 3, 10), (11, 11, 16), (0, 0, 6), (15, 15, 15), (5, 5, 14), (0, 0, 15), (1

127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -
127.0.0.1 - - [04/Jan/2023 13:54:48] "POST / HTTP/1.1" 200 -



[(19, 19, 14), (1, 1, 16), (13, 13, 4), (8, 8, 6), (13, 13, 13), (2, 2, 5), (8, 8, 15), (2, 2, 14), (11, 11, 5), (5, 5, 3), (3, 3, 2), (5, 5, 12), (15, 15, 7), (3, 3, 11), (19, 19, 7), (0, 0, 7), (1, 1, 0), (19, 19, 16), (15, 15, 16), (0, 0, 16), (1, 1, 9), (13, 13, 6), (1, 1, 18), (8, 8, 8), (13, 13, 15), (2, 2, 7), (8, 8, 17), (2, 2, 16), (5, 5, 5), (3, 3, 4), (19, 19, 0), (0, 0, 0), (3, 3, 13), (19, 19, 9), (15, 15, 9), (0, 0, 9), (1, 1, 2), (19, 19, 18), (15, 15, 18), (0, 0, 18), (1, 1, 11), (8, 8, 1), (13, 13, 8), (2, 2, 0), (8, 8, 10), (2, 2, 9), (0, 0, 14), (5, 5, 7), (15, 15, 2), (3, 3, 6), (19, 19, 2), (0, 0, 2), (3, 3, 15), (19, 19, 11), (15, 15, 11), (1, 1, 4), (0, 0, 11), (13, 13, 1), (8, 8, 3), (2, 2, 2), (8, 8, 12), (2, 2, 11), (5, 5, 0), (15, 15, 4), (3, 3, 8), (19, 19, 4), (11, 11, 14), (0, 0, 4), (3, 3, 17), (15, 15, 13), (0, 0, 13), (8, 8, 5), (2, 2, 4), (3, 3, 1), (11, 11, 7), (15, 15, 6), (3, 3, 10), (11, 11, 16), (0, 0, 6), (15, 15, 15), (5, 5, 14), (0, 0, 15), (1

The solution to the optimization problem is a new allocation of tasks into virtual machines and virtual machines to physical machines. This allocation is represented as a migration map. Unfortunately CloudSim does not implement reallocation of tasks during running simulation so task-virtual machine allocation stays the same after the initial allocation. Thus the migration map describes just virtual machine to physical machine allocation.